# Goal : Scan the 3D predcitions with the Dpo fold database. 
## However, we will only scan the proteins that did not make it into the first version of the scan 

### A. Make the DB
### B. Run Foldseek
### C. Get the hits with good proba
### D. Report the folds
***

# A

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb/Dpo_domains.phagedepo.0805.final.tsv \
/media/concha-eloko/Linux/depolymerase_building

In [ ]:
path_pdb = "/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files"
path_CAZY_db = "/media/concha-eloko/Linux/PhageDEPOdetection/Dpo_RefDB_v2"

with open(f"{path_pdb}/folds_CAZY.tsv", "w") as outfile :
    outfile.write(f"Groups\tFolds\tProtein\n")
    for file in os.listdir(path_pdb) :
        if os.path.isdir(f"{path_pdb}/{file}") == True and len(os.listdir(f"{path_pdb}/{file}"))>0:
            fold = folds_dico[file].replace("_" , " ")
            for pdb_file in os.listdir(f"{path_pdb}/{file}") :
                outfile.write(f"{file}\t{fold}\t{pdb_file}\n")


In [3]:
import os 
path_pdb = "/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files"
path_CAZY_db = "/media/concha-eloko/Linux/PhageDEPOdetection/Dpo_RefDB_v2"

for file in os.listdir(path_pdb) :
    if os.path.isdir(f"{path_pdb}/{file}") == True and len(os.listdir(f"{path_pdb}/{file}"))>0:
        for pdb_file in os.listdir(f"{path_pdb}/{file}") :
            os.system(f"cp {path_pdb}/{file}/{pdb_file} {path_CAZY_db}")

In [ ]:
# Create the database : 

path_pdb_ref = "/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains"

# Generates the database :

foldseek createdb \
/media/concha-eloko/Linux/PhageDEPOdetection/Dpo_RefDB_v2 \
/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db

# Generate index files : 

foldseek createindex \
/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db \ 
/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2

# Move to server :
rsync -avzhe ssh \
/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase
    
rsync -avzhe ssh \
/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase

***
## B  

> Get a list of the failed Dpos

In [12]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

first_batch_results = set()

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" :
            if row["prob"] >= 0.5 :
                first_batch_results.add((row["query"] , fold))
            elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
                first_batch_results.add((row["query"] , fold))
                
first_batch_dpos = {dpo for (dpo,fold) in first_batch_results}



In [14]:
len(first_batch_dpos)
first_batch_dpos

{'1013.pdb',
 '1020.pdb',
 '1022.pdb',
 '1035.pdb',
 '1064.pdb',
 '1065.pdb',
 '1091.pdb',
 '1095.pdb',
 '11.pdb',
 '1106.pdb',
 '1107.pdb',
 '1108.pdb',
 '1110.pdb',
 '1111.pdb',
 '1112.pdb',
 '1122.pdb',
 '1143.pdb',
 '1147.pdb',
 '1162.pdb',
 '1163.pdb',
 '1181.pdb',
 '1203.pdb',
 '1214.pdb',
 '1290.pdb',
 '134.pdb',
 '1343.pdb',
 '1344.pdb',
 '1401.pdb',
 '141.pdb',
 '1420.pdb',
 '1424.pdb',
 '1429.pdb',
 '1431.pdb',
 '1436.pdb',
 '1437.pdb',
 '144.pdb',
 '1449.pdb',
 '1450.pdb',
 '1454.pdb',
 '146.pdb',
 '1461.pdb',
 '1462.pdb',
 '1463.pdb',
 '1470.pdb',
 '1471.pdb',
 '1477.pdb',
 '1492.pdb',
 '1493.pdb',
 '1494.pdb',
 '1501.pdb',
 '1502.pdb',
 '1503.pdb',
 '1505.pdb',
 '1506.pdb',
 '1507.pdb',
 '1513.pdb',
 '1514.pdb',
 '1515.pdb',
 '1516.pdb',
 '1517.pdb',
 '1519.pdb',
 '1520.pdb',
 '1521.pdb',
 '1523.pdb',
 '1525.pdb',
 '1527.pdb',
 '1531.pdb',
 '1532.pdb',
 '1533.pdb',
 '1535.pdb',
 '1536.pdb',
 '154.pdb',
 '1557.pdb',
 '1559.pdb',
 '1563.pdb',
 '1566.pdb',
 '1568.pdb',
 '1569

In [ ]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/depolymerase_building"
path_pdb = f"{path_project}/phagedepo_out"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/CAZY_seekfold"
    protein_id = path_in.split("/")[-1].split(".pdb")[0]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    #seek = f"foldseek easy-search {path_in} {path_db} {path_out}.html {path_tmp} --format-mode 3"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)

paths = [f"{path_pdb}/{file}" for file in os.listdir(f"{path_pdb}") if file not in first_batch_dpos]


for path in paths :
    seek_beta_helix(path)

***
## C.

In [20]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"
outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

depo_ppt = {}
very_good = 0
good = 0
decent = 0 
passable = 0
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    try : 
        if results_df["prob"][0] > 0.1 :
            depo_ppt[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
            if results_df["prob"][0] > 0.9 :
                very_good += 1
            elif results_df["prob"][0] > 0.5 :
                good += 1
            elif results_df["prob"][0] > 0.25 :
                decent += 1
            else :
                passable += 1
                
    except IndexError :
        pass

import pprint
#pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)


print(f"So far, out of {len((outputs))} proteins searched, {len(depo_ppt)} seemed to be positive in the phagedepo dataset ({len(depo_ppt)/len(outputs)} %).\nThe proportion of very good is {very_good/len(depo_ppt)}\nThe proportion of good is {good/len(depo_ppt)}\nThe proportion of decent is {decent/len(depo_ppt)}\nThe rest is {passable/len(depo_ppt)}")


So far, out of 990 proteins searched, 536 seemed to be positive in the phagedepo dataset (0.5414141414141415 %).
The proportion of very good is 0.332089552238806
The proportion of good is 0.166044776119403
The proportion of decent is 0.13432835820895522
The rest is 0.3675373134328358


In [32]:
path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

info_df["Folds"].unique()

array(['parallel β-helix', 'β-jelly roll', '( α / α ) 6 barrel',
       '( α / α ) 6', '( β / α ) 8 barrel', '6-fold β-propeller',
       'β-propeller', '( α / α ) 6 toroid + anti-parallel β-sheet',
       'lysozyme fold', '5-fold β-propeller', '5 fold β-propeller',
       '( α / α ) 7 barrel', '6-stranded β-barrel',
       'β-sandwich + β-sheet', '6-bladed β-propeller',
       '( α / α ) 3 barrel', '( β / α ) 7 barrel',
       '( β / α ) 8 (inferred)', 'mannosidase fold'], dtype=object)

***
## C.

In [1]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"

path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
depo_results = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        if row["prob"] >= 0.5 :
            try : 
                fold = info_df[info_df["Protein"] == row["target"]]["Folds"].values[0]
                if fold != "parallel β-helix" :
                    if row["alnlen"] >= 200 :
                        print(row["target"] , row["query"], fold)
                        break
            except Exception as e :
                pass

4yzp.pdb 2641.pdb ( β / α ) 8 barrel
2y8k.pdb 738.pdb ( β / α ) 8 barrel
5awo.pdb 4674.pdb ( β / α ) 8 barrel
5bx3.pdb 5536.pdb ( α / α ) 6 barrel
2w5n.pdb 1021.pdb 6-bladed β-propeller
3pe7.pdb 5938.pdb β-propeller
4yzp.pdb 12.pdb ( β / α ) 8 barrel
2w5n.pdb 2588.pdb 6-bladed β-propeller
1nc5.pdb 3125.pdb ( α / α ) 6 barrel
4yzp.pdb 6542.pdb ( β / α ) 8 barrel
4nov.pdb 660.pdb 5-fold β-propeller
5bx3.pdb 3302.pdb ( α / α ) 6 barrel
4yzp.pdb 4095.pdb ( β / α ) 8 barrel
5mqs.pdb 6793.pdb 5-fold β-propeller
1ug9.pdb 5351.pdb ( α / α ) 6 barrel
4yzp.pdb 1086.pdb ( β / α ) 8 barrel
5mqr.pdb 6188.pdb 5-fold β-propeller
5yj6.pdb 5728.pdb ( α / α ) 6 barrel
4yzp.pdb 3327.pdb ( β / α ) 8 barrel
5awo.pdb 5047.pdb ( β / α ) 8 barrel
5awo.pdb 5332.pdb ( β / α ) 8 barrel
3pe7.pdb 4892.pdb β-propeller
2y8k.pdb 1408.pdb ( β / α ) 8 barrel
1ug9.pdb 5365.pdb ( α / α ) 6 barrel
5mqs.pdb 158.pdb 5-fold β-propeller
7nde.pdb 6360.pdb β-jelly roll
6r3r.pdb 3123.pdb 5-fold β-propeller
1vd5.pdb 3294.pdb ( α 

In [ ]:
β / α : 67
α / α : 39

In [6]:
dico_folds["6-bladed beta-propeller"]

['2641.pdb',
 '1343.pdb',
 '1021.pdb',
 '5938.pdb',
 '1020.pdb',
 '1436.pdb',
 '5567.pdb',
 '16.pdb',
 '4299.pdb',
 '6707.pdb',
 '2588.pdb',
 '1848.pdb',
 '6542.pdb',
 '660.pdb',
 '3237.pdb',
 '3706.pdb',
 '5777.pdb',
 '5126.pdb',
 '1695.pdb',
 '11.pdb',
 '2591.pdb',
 '5590.pdb',
 '5724.pdb',
 '4229.pdb',
 '1721.pdb',
 '3944.pdb',
 '5790.pdb',
 '3151.pdb',
 '4892.pdb',
 '3754.pdb',
 '3123.pdb',
 '5343.pdb',
 '3557.pdb',
 '5386.pdb',
 '4178.pdb',
 '785.pdb',
 '5278.pdb',
 '4151.pdb',
 '6548.pdb',
 '2635.pdb',
 '2667.pdb',
 '6230.pdb',
 '2786.pdb',
 '5556.pdb',
 '2011.pdb',
 '3498.pdb',
 '5396.pdb',
 '6332.pdb',
 '5229.pdb',
 '657.pdb',
 '1109.pdb',
 '3857.pdb',
 '6552.pdb',
 '6693.pdb',
 '369.pdb',
 '5631.pdb',
 '3174.pdb',
 '1985.pdb',
 '6508.pdb',
 '51.pdb',
 '1043.pdb',
 '319.pdb',
 '6550.pdb',
 '1344.pdb',
 '3862.pdb',
 '1986.pdb',
 '5589.pdb',
 '814.pdb',
 '2753.pdb',
 '3166.pdb',
 '5629.pdb',
 '5791.pdb',
 '2602.pdb',
 '6046.pdb',
 '183.pdb',
 '732.pdb',
 '2764.pdb',
 '221.pdb',
 

In [6]:
info_df[info_df["X_Group_Name"] == "TIM beta/alpha-barrel" ]

,Activity,ID,X_Group_Name
7,Chitinase II,AF-A0A089GJA0-F1-model_v4_A_2_186_447.pdb,TIM beta/alpha-barrel
16,Chitinase II,AF-A0A166VHV0-F1-model_v4.pdb,TIM beta/alpha-barrel
19,Chitinase II,AF-A0A202B6A7-F1-model_v4.pdb,TIM beta/alpha-barrel
36,Hyaluronidase,AF-C0HLL5-F1-model_v4.pdb,TIM beta/alpha-barrel
40,Hyaluronidase,AF-P49370-F1-model_v4.pdb,TIM beta/alpha-barrel
41,Hyaluronidase,AF-P49371-F1-model_v4.pdb,TIM beta/alpha-barrel


In [5]:
info_df["X_Group_Name"].unique()

array(['jelly-roll', 'alpha/alpha toroid', 'right-handed beta-helix',
       'TIM beta/alpha-barrel', '6-bladed beta-propeller',
       'Alpha/Beta hydrolase fold', 'Other', 'Flavodoxin-like'],
      dtype=object)

In [12]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
path_pdb = f"{path_project}/phagedepo_out"

import pandas as pd
import os 

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]


paths_decipher = [] 
good_phagedepo = {}
decipher_phagedepo = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    try : 
        if results_df["prob"][0] >= 0.5 :
            good_phagedepo[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
        elif results_df["prob"][0] >= 0.25 :
            decipher_phagedepo[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
            paths_decipher.append(f"{path_pdb}/{results_df['query'][0]}")
    except Exception as e :
        print(e)
        pass


index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


In [7]:
pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)

pp.pprint(decipher_phagedepo)

{'1043.pdb': ('AF-A0A202B6A7-F1-model_v4.pdb', 0.353),
 '1125.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.353),
 '1574.pdb': ('A1l_00058_A_1_129_274.pdb', 0.473),
 '1810.pdb': ('AF-A0A0A6QEK9-F1-model_v4.pdb', 0.411),
 '1842.pdb': ('AF-A0A7K3HGV9-F1-model_v4.pdb', 0.353),
 '1873.pdb': ('AF-A0A0F6AK51-F1-model_v4.pdb', 0.277),
 '1918.pdb': ('AF-A0A0D3A0D6-F1-model_v4.pdb', 0.326),
 '2001.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.442),
 '2048.pdb': ('A1l_00058_A_1_129_274.pdb', 0.411),
 '2170.pdb': ('7fhv.pdb_B', 0.277),
 '2190.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.382),
 '2217.pdb': ('AF-A0A1E5LEG1-F1-model_v4.pdb', 0.473),
 '2588.pdb': ('AF-A0A7W9JCS6-F1-model_v4_A_1_95_273.pdb', 0.473),
 '2617.pdb': ('AF-A0A562WF65-F1-model_v4.pdb', 0.326),
 '2641.pdb': ('AF-A0A7G2DEW9-F1-model_v4.pdb', 0.254),
 '2691.pdb': ('AF-A0A0A6QEK9-F1-model_v4.pdb', 0.442),
 '2753.pdb': ('AF-A0A7C4BP91-F1-model_v4.pdb', 0.473),
 '2786.pdb': ('AF-A0A7G2DEW9-F1-model_v4.pdb', 0

In [ ]:
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        if row["query"] 
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if row["prob"] >= 0.2 :
            if row["query"] not in dico_folds[fold] :
                dico_folds[fold].append(row["query"])
        elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
            if row["query"] not in dico_folds[fold] :
                dico_folds[fold].append(row["query"])
                
for fold in dico_folds : 
    print(f"The {fold} presented {len(dico_folds[fold])} depolymerases.\n")

***
# Make the CAZY DB